## 더미 데이터 생성 방식

NH_CONTEST_STK_DT_QUT.csv에서 bse_dt, tck_iem_cd, iem_end_pr 열을 가져옴

필요한 열은 user_id, 종목 종류, 매수 시점, 매도 시점, MBTI 사용 시점, MBTI 사용 시점 총 자산

user_id: 
* 같은 user_id라도 동일한 종목 종류를 1개 이상 가질 수 있음 

종목 종류: 
* tck_iem_cd에 해당하는 모든 종목이 올 수 있음

매수 시점:
* 매도 시점, MBTI 사용 시점과 같거나 무조건 이전 시점이어야 함

매도 시점:
* 매수 시점과 같거나 그 이후여야 함
* MBTI 사용 시점과 같거나 그 이전이어야 함
* NA값이 존재할 수 있음 

MBTI 사용 시점:
* 매수 시점, 매도 시점과 같거나 그 이후여야 함


MBTI 사용 시점 총 자산:
* 매도되지 않은 종목의 종목 종가를 MBTI 사용 시점에 맞는 bse_dt기준으로 가져옴
* 이렇게 가져온 종목 종가 금액을 모두 합산한 가격이 총 자산이 됨

### 더미 데이터 생성 코드 ver2

In [ ]:
import pandas as pd
import random
import numpy as np

# 주어진 경로에서 기존 데이터를 로드
csv_path = r"C:/Users/asdjj/OneDrive/바탕 화면/코딩 공부/NH 데이콘/데이터/NH_CONTEST_STK_DT_QUT.csv"
csv_data = pd.read_csv(csv_path)

# 'tck_iem_cd' 열에서 공백을 제거하여 정리
csv_data['tck_iem_cd'] = csv_data['tck_iem_cd'].str.strip()

# 고유한 종목 코드(tck_iem_cd) 목록을 가져옴
stock_types = csv_data['tck_iem_cd'].unique()

# 더미 데이터를 생성
user_ids = [f"user_{i}" for i in range(1, 101)]
date_range = pd.to_datetime(csv_data['bse_dt'].astype(str), format='%Y%m%d').sort_values().unique()

# 무작위로 더미 데이터를 생성하는 함수를 정의
def create_dummy_data(user_ids, stock_types, date_range, csv_data):
    data = []
    
    for user_id in user_ids:
        # 각 사용자가 매수할 주식의 수를 무작위로 선택
        num_stocks = random.randint(1, len(stock_types))
        chosen_stocks = random.sample(list(stock_types), num_stocks)
        
        # MBTI 사용 시점을 무작위로 선택
        mbti_time = random.choice(date_range)
        
        # 각 선택된 주식에 대해 무작위 매수/매도/MBTI 사용 시점을 생성
        for stock in chosen_stocks:
            buy_time = random.choice(date_range[date_range <= mbti_time])
            # 매도 시점이 NA일 확률을 50%로 설정
            if random.random() > 0.5:
                sell_time = random.choice(date_range[(date_range >= buy_time) & (date_range <= mbti_time)])
            else:
                sell_time = np.nan
            
            data.append([user_id, stock, buy_time, sell_time, mbti_time])
    
    dummy_df = pd.DataFrame(data, columns=['user_id', '종목 종류', '매수 시점', '매도 시점', 'MBTI 사용 시점'])
    return dummy_df

# 더미 데이터를 생성
dummy_data = create_dummy_data(user_ids, stock_types, date_range, csv_data)

# MBTI 사용 시점 총 자산을 계산하는 함수
def calculate_total_assets(df, csv_data):
    total_assets_dict = {}
    
    # 각 사용자에 대해 매도 시점이 NA인 행을 찾음
    for user_id in df['user_id'].unique():
        user_data = df[df['user_id'] == user_id]
        unsold_stocks = user_data[user_data['매도 시점'].isna()]
        
        if not unsold_stocks.empty:
            total_assets = 0
            mbti_time = unsold_stocks['MBTI 사용 시점'].iloc[0]
            
            for _, row in unsold_stocks.iterrows():
                stock = row['종목 종류']
                mbti_date = mbti_time.strftime('%Y%m%d')
                stock_price = csv_data[(csv_data['tck_iem_cd'] == stock) & (csv_data['bse_dt'] == int(mbti_date))]['iem_end_pr']
                
                if not stock_price.empty:
                    total_assets += stock_price.iloc[0]
            
            total_assets_dict[user_id] = total_assets
        else:
            total_assets_dict[user_id] = 0
    
    # MBTI 사용 시점 총 자산을 새로운 열에 추가
    df['MBTI 사용 시점 총 자산'] = df['user_id'].map(total_assets_dict)
    
    return df

# 총 자산을 계산
dummy_data_with_assets = calculate_total_assets(dummy_data, csv_data)


dummy_data_with_assets.head()


In [ ]:
dummy_data_with_assets.head(100)

In [ ]:
dummy_data_with_assets.to_csv(r'C:/Users/asdjj/OneDrive/바탕 화면/코딩 공부/NH 데이콘/데이터/재구성 데이터/dummy_data_with_assets.csv', index=False, encoding='utf-8-sig')